In [5]:
import pandas as pd
import pyarrow
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import pickle
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
import mlflow
mlflow.set_tracking_uri("http://localhost:8080")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='mlflow-artifacts:/450387268741673689', creation_time=1699024876986, experiment_id='450387268741673689', last_update_time=1699024876986, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('green_tripdata_2021-01.parquet')
df_val = read_dataframe('green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715205895199

In [21]:
with open('lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [14]:
with mlflow.start_run():

    mlflow.set_tag("developer", "saurabh")

    mlflow.log_param("train-data-path", "green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "green_tripdata_2021-02.csv")

    alpha = 0.02
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)


In [15]:
lr.coef_

array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        0.0000000e+00, -0.0000000e+00,  6.7214058e-05])

In [16]:
import xgboost as xgb

In [17]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=10
            evals=[(valid, 'validation')],
            early_stopping_rounds=5
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3897761367.py, line 8)

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=15,
    trials=Trials()
)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/home/kukreti/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [11:02:06] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.54795                          
[1]	validation-rmse:10.95268                          
[2]	validation-rmse:10.42178                          
[3]	validation-rmse:9.94895                           
[4]	validation-rmse:9.52943                           
[5]	validation-rmse:9.15714                           
[6]	validation-rmse:8.82911                           
[7]	validation-rmse:8.53971                           
[8]	validation-rmse:8.28601                           
[9]	validation-rmse:8.06283                           
 10%|█         | 1/10 [00:21<03:15, 21.77s/trial, best loss: 8.062831508670556]

/home/kukreti/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [11:02:27] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.59662                                                   
[1]	validation-rmse:9.42126                                                    
[2]	validation-rmse:8.57783                                                    
[3]	validation-rmse:7.98452                                                    
[4]	validation-rmse:7.57552                                                    
[5]	validation-rmse:7.29421                                                    
[6]	validation-rmse:7.09671                                                    
[7]	validation-rmse:6.95947                                                    
[8]	validation-rmse:6.86364                                                    
[9]	validation-rmse:6.79331                                                    
 20%|██        | 2/10 [00:39<02:36, 19.53s/trial, best loss: 6.793309629135656]

...

[0]	validation-rmse:9.49020                                                   
[1]	validation-rmse:8.07056                                                   
[2]	validation-rmse:7.37224                                                   
[3]	validation-rmse:7.02779                                                   
[4]	validation-rmse:6.85920                                                   
[5]	validation-rmse:6.77175                                                   
[6]	validation-rmse:6.72460                                                   
[7]	validation-rmse:6.69626                                                   
[8]	validation-rmse:6.67324                                                   
[9]	validation-rmse:6.66327                                                   
 80%|████████  | 8/10 [02:53<00:35, 17.93s/trial, best loss: 6.58835819291664]

/home/kukreti/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [11:04:59] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.17152                                                   
[1]	validation-rmse:7.70923                                                   
[2]	validation-rmse:7.05367                                                   
[3]	validation-rmse:6.75787                                                   
[4]	validation-rmse:6.62065                                                   
[5]	validation-rmse:6.55041                                                   
[6]	validation-rmse:6.50841                                                   
[7]	validation-rmse:6.48216                                                   
[8]	validation-rmse:6.46679                                                   
[9]	validation-rmse:6.45868                                                   
 90%|█████████ | 9/10 [03:10<00:17, 17.71s/trial, best loss: 6.458675110695369]

/home/kukreti/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [11:05:16] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.71519                                                   
[1]	validation-rmse:11.25831                                                   
[2]	validation-rmse:10.83789                                                   
[3]	validation-rmse:10.44996                                                   
[4]	validation-rmse:10.09773                                                   
[5]	validation-rmse:9.77264                                                    
[6]	validation-rmse:9.47752                                                    
[7]	validation-rmse:9.20683                                                    
[8]	validation-rmse:8.96025                                                    
[9]	validation-rmse:8.73525                                                    
100%|██████████| 10/10 [04:20<00:00, 26.06s/trial, best loss: 6.458675110695369]


In [24]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

RestException: RESOURCE_DOES_NOT_EXIST: Could not find experiment with ID 450387268741673689